In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_core.messages import AIMessage, HumanMessage, trim_messages
from langgraph.graph import MessagesState, StateGraph, START, END
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

def chat_model_node(state: MessagesState):
    """Trim messages to last 100 tokens before sending to LLM"""
    trimmed_messages = trim_messages(
        state["messages"],
        max_tokens=100,  # Only keep last 100 tokens worth of messages
        strategy="last",  # Keep the most recent messages
        token_counter=ChatOpenAI(model="gpt-4o"),
        allow_partial=False  # Don't cut messages in half
    )
    return {"messages": [llm.invoke(trimmed_messages)]}


In [3]:
builder = StateGraph(MessagesState)
builder.add_node("chat_model", chat_model_node)
builder.add_edge(START, "chat_model")
builder.add_edge("chat_model", END)
graph = builder.compile()

In [4]:
messages = [
    HumanMessage("Hi! I'm learning about dogs.", name="User"),
    AIMessage("Great! Dogs are wonderful pets.", name="Bot"),
    HumanMessage("What breeds are best for families?", name="User"),
    AIMessage("Golden Retrievers and Labradors are excellent family dogs.", name="Bot"),
    HumanMessage("Tell me about their temperament.", name="User")
]


ERROR! Session/line number was not unique in database. History logging moved to new session 83


In [6]:
print("ORIGINAL CONVERSATION:")
print(f"Total messages: {len(messages)}\n")
for message in messages:
    message.pretty_print()


ORIGINAL CONVERSATION:
Total messages: 5

================================ Human Message =================================
Name: User

Hi! I'm learning about dogs.
================================== Ai Message ==================================
Name: Bot

Great! Dogs are wonderful pets.
================================ Human Message =================================
Name: User

What breeds are best for families?
================================== Ai Message ==================================
Name: Bot

Golden Retrievers and Labradors are excellent family dogs.
================================ Human Message =================================
Name: User

Tell me about their temperament.


In [7]:
trimmed = trim_messages(
    messages,
    max_tokens=100,
    strategy="last",
    token_counter=ChatOpenAI(model="gpt-4o"),
    allow_partial=False
)

print("\n" + "="*60)
print("TRIMMED MESSAGES (what LLM actually sees):")
for message in trimmed:
    message.pretty_print()


TRIMMED MESSAGES (what LLM actually sees):
================================ Human Message =================================
Name: User

Hi! I'm learning about dogs.
================================== Ai Message ==================================
Name: Bot

Great! Dogs are wonderful pets.
================================ Human Message =================================
Name: User

What breeds are best for families?
================================== Ai Message ==================================
Name: Bot

Golden Retrievers and Labradors are excellent family dogs.
================================ Human Message =================================
Name: User

Tell me about their temperament.


In [8]:
output = graph.invoke({'messages': messages})

In [10]:
for m in output['messages']:
    m.pretty_print()

================================ Human Message =================================
Name: User

Hi! I'm learning about dogs.
================================== Ai Message ==================================
Name: Bot

Great! Dogs are wonderful pets.
================================ Human Message =================================
Name: User

What breeds are best for families?
================================== Ai Message ==================================
Name: Bot

Golden Retrievers and Labradors are excellent family dogs.
================================ Human Message =================================
Name: User

Tell me about their temperament.
================================== Ai Message ==================================

Golden Retrievers and Labrador Retrievers are renowned for their friendly and gentle temperaments, making them ideal family pets.

**Golden Retrievers**:
- **Friendly and Sociable**: Golden Retrievers are known for their friendliness towards people and other animals.